# Quantum Reservoir Computing (QRC) で時系列2値分類

このノートブックは **Google Colab 実行**を前提に、
- GitHubからリポジトリをclone
- 可視化コードは外部モジュール (`src/spin_viz.py`) をimport
- QRC計算の本体はNotebook内に記述
という構成で作っています。

## 0. Colabセットアップ (GitHub clone + install)

`REPO_URL` をあなたのGitHub URLに変更して実行してください。

In [ ]:
import os
import sys
import pathlib
import subprocess

REPO_URL = "https://github.com/dainfinity/qrc_classification_demo.git"
REPO_DIR = "qrc_classification_demo"

if not os.path.exists(REPO_DIR):
    subprocess.run(["git", "clone", REPO_URL, REPO_DIR], check=True)

os.chdir(REPO_DIR)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", "requirements-colab.txt"], check=True)

repo_root = pathlib.Path.cwd()
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

print("Working directory:", repo_root)

## 1. 問題設定

このデモでは、すでにCSVとして保存された固定長系列 `L` の2値分類を行います。

- `data/train_X.csv`, `data/train_y.csv`
- `data/test_X.csv`, `data/test_y.csv`

各サンプルをQRCに流し、時刻ごとのリザバー状態ベクトルを並べた行列 (\(L\times N\)) を作ります。
testは **1-NN** で分類し、行列間距離は **フロベニウスノルム**を使います。

In [ ]:
import os
import math
from itertools import repeat
from concurrent.futures import ProcessPoolExecutor

import numpy as np
import pandas as pd
from scipy.linalg import expm
import ipywidgets as widgets
from IPython.display import display

SEED = 1234
np.random.seed(SEED)

USE_PARALLEL = True
N_WORKERS = min(4, os.cpu_count() or 2)
print(f"USE_PARALLEL={USE_PARALLEL}, N_WORKERS={N_WORKERS}, SEED={SEED}")

In [ ]:
train_X = pd.read_csv("data/train_X.csv").values.astype(np.float64)
train_y = pd.read_csv("data/train_y.csv")["label"].values.astype(int)
test_X = pd.read_csv("data/test_X.csv").values.astype(np.float64)
test_y = pd.read_csv("data/test_y.csv")["label"].values.astype(int)

M_train, L = train_X.shape
M_test, L2 = test_X.shape
assert L == L2
print(f"Train: {train_X.shape}, Test: {test_X.shape}, sequence length L={L}")

## 2. スピン系の可視化付きハイパーパラメータ設定

下のセルは外部モジュール `src/spin_viz.py` をimportして、
- トポロジー (全結合 / 1D鎖 / 2D近接)
- スピン数 `N`
- 結合幅 `J width`
- 横磁場 `h field`
をスライダー/ドロップダウンで変更できます。

結合色はランダム結合強度、スピン色は横磁場強度に応じて変化します。

In [ ]:
from src.spin_viz import build_spin_control_panel

panel, controls = build_spin_control_panel(default_n=8, seed=SEED)
display(panel)

dt_slider = widgets.FloatSlider(value=0.20, min=0.02, max=1.00, step=0.02, description="dt")
gamma_slider = widgets.FloatSlider(value=1.00, min=0.10, max=3.00, step=0.05, description="input γ")
display(widgets.HBox([dt_slider, gamma_slider]))

print("計算前に上のパラメータを調整してください。")

## 3. QRC本体コード

ハミルトニアンは

\[
H = \sum_{(i,j)} J_{ij} Z_i Z_j + h\sum_i X_i + \gamma u_t Z_0
\]

とし、時刻ごとに
\( U_t = e^{-i H \Delta t} \)
で状態を更新します。
観測量は各サイトの \(<Z_i>\) です。

In [ ]:
SIGMA_X = np.array([[0, 1], [1, 0]], dtype=np.complex128)
SIGMA_Z = np.array([[1, 0], [0, -1]], dtype=np.complex128)
IDENTITY = np.eye(2, dtype=np.complex128)

def kron_many(ops):
    out = np.array([[1.0 + 0.0j]])
    for op in ops:
        out = np.kron(out, op)
    return out

def site_operator(n_spins, site, op):
    mats = [IDENTITY] * n_spins
    mats[site] = op
    return kron_many(mats)

def zz_operator(n_spins, i, j):
    mats = [IDENTITY] * n_spins
    mats[i] = SIGMA_Z
    mats[j] = SIGMA_Z
    return kron_many(mats)

def grid_shape(n_spins):
    rows = int(np.floor(np.sqrt(n_spins)))
    while rows > 1 and n_spins % rows != 0:
        rows -= 1
    cols = int(np.ceil(n_spins / rows))
    return rows, cols

def build_edges(n_spins, topology):
    if topology == "all_to_all":
        return [(i, j) for i in range(n_spins) for j in range(i + 1, n_spins)]
    if topology == "chain_1d":
        return [(i, i + 1) for i in range(n_spins - 1)]
    if topology == "grid_2d":
        rows, cols = grid_shape(n_spins)
        edges = []
        for idx in range(n_spins):
            r, c = divmod(idx, cols)
            right = idx + 1
            down = idx + cols
            if c + 1 < cols and right < n_spins:
                edges.append((idx, right))
            if r + 1 < rows and down < n_spins:
                edges.append((idx, down))
        return edges
    raise ValueError(f"Unknown topology: {topology}")

def build_reservoir_operators(n_spins):
    x_ops = [site_operator(n_spins, i, SIGMA_X) for i in range(n_spins)]
    z_ops = [site_operator(n_spins, i, SIGMA_Z) for i in range(n_spins)]
    return x_ops, z_ops

def build_static_hamiltonian(n_spins, topology, j_width, h_field, seed=SEED):
    x_ops, z_ops = build_reservoir_operators(n_spins)
    edges = build_edges(n_spins, topology)
    rng = np.random.default_rng(seed)

    dim = 2 ** n_spins
    h_int = np.zeros((dim, dim), dtype=np.complex128)
    for (i, j) in edges:
        jij = rng.uniform(-j_width, j_width)
        h_int += jij * zz_operator(n_spins, i, j)

    h_field_term = np.zeros((dim, dim), dtype=np.complex128)
    for i in range(n_spins):
        h_field_term += h_field * x_ops[i]

    h0 = h_int + h_field_term
    return h0, z_ops

def sequence_to_state_matrix(sequence, params):
    n_spins = params["n_spins"]
    topology = params["topology"]
    j_width = params["j_width"]
    h_field = params["h_field"]
    dt = params["dt"]
    gamma = params["gamma"]
    seed = params["seed"]

    h0, z_ops = build_static_hamiltonian(n_spins, topology, j_width, h_field, seed=seed)
    z_input = z_ops[0]

    dim = 2 ** n_spins
    psi = np.zeros((dim,), dtype=np.complex128)
    psi[0] = 1.0 + 0.0j

    states = np.zeros((len(sequence), n_spins), dtype=np.float64)
    for t, u in enumerate(sequence):
        h_t = h0 + gamma * float(u) * z_input
        u_t = expm(-1j * h_t * dt)
        psi = u_t @ psi

        for i in range(n_spins):
            states[t, i] = np.real(np.vdot(psi, z_ops[i] @ psi))

    return states

def build_matrices_for_dataset(X, params, use_parallel=True, n_workers=2):
    if use_parallel:
        try:
            with ProcessPoolExecutor(max_workers=n_workers) as ex:
                mats = list(ex.map(sequence_to_state_matrix, X, repeat(params)))
            return np.stack(mats, axis=0)
        except Exception as e:
            print(f"Parallel execution failed ({e}), fallback to sequential.")

    mats = [sequence_to_state_matrix(seq, params) for seq in X]
    return np.stack(mats, axis=0)

def predict_1nn_frobenius(train_mats, train_labels, test_mats):
    preds = []
    for test_mat in test_mats:
        diffs = train_mats - test_mat[None, :, :]
        dists = np.linalg.norm(diffs, ord='fro', axis=(1, 2))
        nn_idx = int(np.argmin(dists))
        preds.append(int(train_labels[nn_idx]))
    return np.array(preds, dtype=int)

## 4. パラメータ確定 → リザバー行列作成 → 1-NN分類

- 可視化スライダーの現在値を読み取り
- train/testそれぞれでリザバー状態行列を作成
- `.npy` として保存
- フロベニウスノルム1-NNで評価

In [ ]:
reservoir_params = {
    "n_spins": int(controls["n_spins"].value),
    "topology": controls["topology"].value,
    "j_width": float(controls["j_width"].value),
    "h_field": float(controls["h_field"].value),
    "dt": float(dt_slider.value),
    "gamma": float(gamma_slider.value),
    "seed": SEED,
}
print("Reservoir params:")
for k, v in reservoir_params.items():
    print(f"  {k}: {v}")

train_mats = build_matrices_for_dataset(
    train_X, reservoir_params, use_parallel=USE_PARALLEL, n_workers=N_WORKERS
)
test_mats = build_matrices_for_dataset(
    test_X, reservoir_params, use_parallel=USE_PARALLEL, n_workers=N_WORKERS
)

np.save("data/train_reservoir_matrices.npy", train_mats)
np.save("data/test_reservoir_matrices.npy", test_mats)
print("saved: data/train_reservoir_matrices.npy")
print("saved: data/test_reservoir_matrices.npy")

pred = predict_1nn_frobenius(train_mats, train_y, test_mats)
acc = (pred == test_y).mean()
print(f"1-NN accuracy (Frobenius): {acc:.3f}")

## 5. 相談ポイント (次の改善)

次に相談しやすい拡張例:
1. 入力埋め込みの改善 (複数サイト駆動や非線形変換)
2. 観測量の拡張 (\(<X_i>\), \(<Y_i>\), 相関 \(<Z_i Z_j>\))
3. データタスク変更 (より難しい2値分類、ノイズ強化、長系列化)